**Library Install & Imports**

In [ ]:
#!pip install geopandas
#!pip install -U geemap

In [ ]:
import ee
import geemap
geemap.update_package()
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

Unzipping geemap-master.zip ...
Data downloaded to: /root/Downloads/geemap-master

Please comment out 'geemap.update_package()' and restart the kernel to take effect:
Jupyter menu -> Kernel -> Restart & Clear Output


In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-jayantakurp17')

**Loading the Map**

In [ ]:
Map = geemap.Map(center=(35.117752412600176, -89.93545863387348), zoom=9)
Map

Map(center=[35.117752412600176, -89.93545863387348], controls=(WidgetControl(options=['position', 'transparent…

**Landsat Image Call**

In [ ]:
aoi = ee.FeatureCollection("projects/ee-jayantakurp17/assets/shelby_county").geometry()
Map.addLayer(aoi, {}, 'Shelby County')

In [ ]:
def maskL8sr(image):
    # Select the 'pixel_qa' band for Landsat 8 SR
    qa = image.select('QA_PIXEL')
    # Create a mask for cloud-free pixels
    cloud_mask = qa.bitwiseAnd(1 << 5).eq(0)  # Bit 5 is for clouds
    shadow_mask = qa.bitwiseAnd(1 << 3).eq(0)  # Bit 3 is for shadows
    return image.updateMask(cloud_mask).updateMask(shadow_mask)


In [ ]:
# Load Landsat 8 image collection for Summer
summer = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
             .filterBounds(aoi) \
             .filterDate('2022-06-01', '2022-08-30') \
             .filterMetadata('CLOUD_COVER', 'less_than', 0.1) \


summer8 = summer.max().clip(aoi)

In [ ]:
Map.addLayer(summer8, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'Summer')

**Define Function for Indices**

*NDVI*

In [ ]:
# Function to calculate NDVI
def calculate_ndvi(image):
    nir = image.select('B5')
    red = image.select('B4')
    ndvi = image.expression(
        '(NIR-RED)/(NIR+RED)',
        {
            'NIR': nir,
            'RED': red
        }
    ).rename('NDVI')
    return ndvi

*SAVI*

In [ ]:
# Function to calculate SAVI
def calculate_savi(image):
    nir = image.select('B5')
    red = image.select('B4')
    savi = image.expression(
        '((NIR-RED)/(NIR+RED+0.5))*(1.5)',
        {
            'NIR': nir,
            'RED': red
        }
    ).rename('SAVI')
    return savi

*MSAVI*

In [ ]:
# Function to calculate MSAVI
def calculate_msavi(image):
    nir = image.select('B5')
    red = image.select('B4')
    msavi = image.expression(
        '((2 * NIR + 1) - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2',
        {
            'NIR': nir,
            'RED': red
        }
    ).rename('MSAVI')
    return msavi

*GRVI: Green Vegetation Index*

In [ ]:
# Function to calculate GRVI
def calculate_grvi(image):
    green = image.select('B3')
    red = image.select('B4')
    grvi = image.expression(
        '(GREEN-RED)/(GREEN+RED)',
        {
            'GREEN': green,
            'RED': red
        }
    ).rename('GRVI')
    return grvi

*NDWI*

In [ ]:
# Function to calculate NDWI
def calculate_ndwi(image):
    nir = image.select('B5')
    green = image.select('B3')
    ndwi = image.expression(
        '(GREEN-NIR)/(NIR+GREEN)',
        {
            'NIR': nir,
            'GREEN': green
        }
    ).rename('NDWI')
    return ndwi

*MNDWI*

In [ ]:
# Function to calculate MNDWI
def calculate_mndwi(image):
    swir = image.select('B6')
    green = image.select('B3')
    mndwi = image.expression(
        '(GREEN-SWIR)/(SWIR+GREEN)',
        {
            'SWIR': swir,
            'GREEN': green
        }
    ).rename('MNDWI')
    return mndwi

*NDMI*

In [ ]:
# Function to calculate NDMI
def calculate_ndmi(image):
    nir = image.select('B5')
    swir = image.select('B6')
    ndmi = image.expression(
        '(NIR-SWIR)/(NIR+SWIR)',
        {
            'NIR': nir,
            'SWIR': swir
        }
    ).rename('NDMI')
    return ndmi

*MAWEI*

In [ ]:
# Function to calculate MAWEI
def calculate_mawei(image):
    nir = image.select('B5')
    red = image.select('B4')
    green = image.select('B3')
    blue = image.select('B2')
    swir2 = image.select('B7')
    mawei = image.expression(
        '(5)*(GREEN-NIR)+(BLUE+RED-4*SWIR2)',
        {
            'NIR': nir,
            'SWIR2': swir2,
            'RED': red,
            'GREEN': green,
            'BLUE': blue
        }
    ).rename('MAWEI')
    return mawei

*NDBI*

In [ ]:
# Function to calculate NDBI
def calculate_ndbi(image):
    nir = image.select('B5')
    swir = image.select('B6')
    ndbi = image.expression(
        '(SWIR-NIR)/(NIR+SWIR)',
        {
            'NIR': nir,
            'SWIR': swir
        }
    ).rename('NDBI')
    return ndbi

*LST*

In [ ]:
def calculate_lst(image):
    """
    Function to calculate NDVI, FV, Emissivity, and LST from Landsat 8 data using geemap and Earth Engine.

    Args:
    - image (ee.Image): Landsat 8 image.
    - aoi (ee.Geometry): Area of interest (AOI) for calculations.

    Returns:
    - ee.Image: Image containing NDVI, FV, EM, and LST.
    """

    # Step 1: Calculate NDVI
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')

    # Step 2: Calculate minimum and maximum NDVI in the AOI
    ndviMin = ee.Number(ndvi.reduceRegion(
        reducer=ee.Reducer.min(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).values().get(0))

    ndviMax = ee.Number(ndvi.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=aoi,
        scale=30,
        maxPixels=1e9
    ).values().get(0))

    # Step 3: Calculate Fractional Vegetation (FV)
    fv = ndvi.subtract(ndviMin) \
             .divide(ndviMax.subtract(ndviMin)) \
             .pow(ee.Number(2)) \
             .rename('FV')

    # Step 4: Calculate Emissivity (EM)
    em = fv.multiply(ee.Number(0.004)).add(ee.Number(0.986)).rename('EM')

    # Step 5: Select Thermal Band (Band 10)
    thermal = image.select('B10').rename('thermal')

    # Step 6: Calculate Land Surface Temperature (LST)
    lst = thermal.expression(
        '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
            'TB': thermal.select('thermal'),  # Brightness temperature in Kelvin
            'em': em  # Emissivity
        }).rename('LST')
    return lst

**Calculation of Indices**

*Summer*

In [ ]:
ndvi_summer22 = calculate_ndvi(summer8)
savi_summer22 = calculate_savi(summer8)
msavi_summer22 = calculate_msavi(summer8)
grvi_summer22 = calculate_grvi(summer8)
ndwi_summer22 = calculate_ndwi(summer8)
mndwi_summer22 = calculate_mndwi(summer8)
ndmi_summer22 = calculate_ndmi(summer8)
mawei_summer22 = calculate_mawei(summer8)
ndbi_summer22 = calculate_ndbi(summer8)
lst_summer22 = calculate_lst(summer8)

*Export*

In [ ]:
image_ids = [
    ('ndvi_summer22', ndvi_summer22),
    ('savi_summer22', savi_summer22),
    ('msavi_summer22', msavi_summer22),
    ('grvi_summer22', grvi_summer22),
    ('ndwi_summer22', ndwi_summer22),
    ('mndwi_summer22', mndwi_summer22),
    ('ndmi_summer22', ndmi_summer22),
    ('mawei_summer22', mawei_summer22),
    ('ndbi_summer22', ndbi_summer22),
    ('lst_summer22', lst_summer22)
]

In [ ]:
# Loop over the image IDs and export each image using its name
for image_name, image in image_ids:
    # Set export parameters
    export_params = {
        'description': image_name,         # Use the image name as the description
        'folder': 'Blue_Green',            # Google Drive folder
        'scale': 30,                       # Resolution in meters
        'region': aoi,                     # AOI for clipping (optional)
        'fileFormat': 'GeoTIFF',           # Export format
        'maxPixels': 1e13,                 # Max pixels allowed
    }

    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(image, **export_params)

    # Start the export task
    task.start()
    print(f"Exporting {image_name} to Google Drive...")

print("All export tasks started!")


Exporting ndvi_summer22 to Google Drive...
Exporting savi_summer22 to Google Drive...
Exporting msavi_summer22 to Google Drive...
Exporting grvi_summer22 to Google Drive...
Exporting ndwi_summer22 to Google Drive...
Exporting mndwi_summer22 to Google Drive...
Exporting ndmi_summer22 to Google Drive...
Exporting mawei_summer22 to Google Drive...
Exporting ndbi_summer22 to Google Drive...
Exporting lst_summer22 to Google Drive...
All export tasks started!
